In [1]:
import pandas as pd
import os
import sys

In [2]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [3]:
BW_PROJECT = 'metallican' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

Databases dictionary with 5 object(s):
	Regioinvent
	biosphere3
	biosphere3_spatialized_flows
	cutoff310
	cutoff310 regionalized

In [4]:
facility_df = pd.read_excel(r'data/MetalliCan/production_sites.xlsx', sheet_name='facility') 
facility_group_df = pd.read_excel(r'data/MetalliCan/production_sites.xlsx', sheet_name='facility_group')

In [5]:
from constants import CA_provinces

In [6]:
# Create a new database
metallican_db = bd.Database("MetalliCan")

# Create LCIs for each facility and facility group

In [7]:
def build_lci_entries(df, facility_col, site_id_col, db_name='MetalliCan'):
    lcis = {}
    for _, row in df.iterrows():
        name = f"{row['mining_processing_type']} {row['commodity']} {row['facility_type']}, {row[facility_col]}".strip()
        location = CA_provinces.get(row['province'], 'CA')
        site_id = row[site_id_col]
        key = (db_name, name)
        lcis[key] = {
            'name': name,
            'unit': 'kilogram',
            'location': location,
            'exchanges': [],
            'type': 'process',
            'description': f"This is a site-specific LCI drawn from the MetalliCan database. Site ID is {site_id}."
        }
    return lcis

In [9]:
# Create LCI metadata from both facility and facility_group
facility_lcis = build_lci_entries(facility_df, "facility_name", "main_id")
facility_group_lcis = build_lci_entries(facility_group_df, "facility_group_name", "facility_group_id")

In [12]:
metallican_db.write(facility_lcis)

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/29/2025 14:41:53
  Finished: 07/29/2025 14:41:53
  Total time elapsed: 00:00:00
  CPU %: 104.20
  Memory %: 1.16


In [11]:
metallican_db.write(facility_group_lcis)

Writing activities to SQLite3 database:
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/29/2025 14:41:21
  Finished: 07/29/2025 14:41:21
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.16


# Populate the LCIs with MetalliCan entries

In [ ]:
def populate_lci_with_exchanges(lcis, data_table, mapping_table, site_id_column='site_id'):
    """
    Populates each LCI entry in `lcis` with exchanges from `data_table`, using `mapping_table`.
    
    Parameters:
        lcis (dict): Dictionary of LCI entries keyed by (database_name, process_name).
        data_table (pd.DataFrame): Raw data table with flow values and site IDs.
        mapping_table (pd.DataFrame): Table with mapping: internal name -> (db, code, unit, type).
        site_id_column (str): Column in data_table that links to each site ID.
    """
    for key, process in lcis.items():
        # Extract site ID from the process description or a metadata field you added earlier
        site_id = None
        if 'description' in process and 'Site ID is ' in process['description']:
            site_id = process['description'].split('Site ID is ')[-1].strip('. ')
        
        if site_id is None:
            continue

        # Filter data for this specific site
        site_data = data_table[data_table[site_id_column] == site_id]

        # Build exchanges
        for _, row in site_data.iterrows():
            internal_name = row['flow_name']  # or column that matches your mapping
            amount = row['amount']

            # Find mapping
            try:
                mapped = mapping_table[mapping_table['internal_name'] == internal_name].iloc[0]
            except IndexError:
                continue  # skip if not found

            exchange = {
                'input': (mapped['database'], mapped['code']),
                'amount': amount,
                'unit': mapped['unit'],
                'type': mapped['flow_type']
            }

            process['exchanges'].append(exchange)


# Allocation